<a href="https://colab.research.google.com/github/soroushmosavati/MMAI/blob/main/Leaves_Disease_SM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environement Setup

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import cv2 as cv
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

import tensorflow_hub as hub
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, Activation

import zipfile



Mounted at /content/gdrive


# Data Prepration


In [ ]:
#Loading Dataset and listing Category labels
data_dir = "/content/gdrive/MyDrive/Colab Notebooks/tea sickness dataset/"
class_names = os.listdir(data_dir)
class_names

['algal leaf',
 'white spot',
 'healthy',
 'gray light',
 'bird eye spot',
 'Anthracnose',
 'red leaf spot',
 'brown blight']

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [ ]:
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(256, 256),
    batch_size=32,
    validation_split=0.2,
    class_mode='categorical')


TypeError: ignored

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 885 files belonging to 8 classes.
Using 177 files for validation.


In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(16, 256, 256, 3)
(16,)


In [ ]:
image_batch = tf.cast(tf.expand_dims(image_batch, 0), tf.float32)

In [ ]:
batch_size = 16
img_height = 256
img_width = 256

train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 885 files belonging to 8 classes.
Using 708 files for training.


In [ ]:
normalization_layer = layers.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


0.008501839 0.99644613


# Build Model

In [ ]:
def build_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(256, 256, 3)), # Define the input shape
        # Define the model architecture
        tf.keras.layers.Conv2D(16, 2, padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(2),
        tf.keras.layers.Conv2D(32, 2, padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(2),
        tf.keras.layers.Conv2D(64, 2, padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(2),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(8)
    ])

    return model
model = build_model()
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 256, 256, 3)       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 256, 256, 16)      208       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 128, 128, 16)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 128, 128, 32)      2080      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 64, 64, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 64, 64, 64)       

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

epochs=20
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/20


45/45 [==============================] - 109s 2s/step - loss: 256.2315 - accuracy: 0.2542 - val_loss: 10.0964 - val_accuracy: 0.3333
Epoch 2/20
45/45 [==============================] - 104s 2s/step - loss: 4.8948 - accuracy: 0.4209 - val_loss: 1.8961 - val_accuracy: 0.4463
Epoch 3/20
45/45 [==============================] - 94s 2s/step - loss: 1.2372 - accuracy: 0.5876 - val_loss: 2.8056 - val_accuracy: 0.4802
Epoch 4/20
45/45 [==============================] - 101s 2s/step - loss: 1.2094 - accuracy: 0.5636 - val_loss: 1.5706 - val_accuracy: 0.5367
Epoch 5/20
45/45 [==============================] - 115s 2s/step - loss: 0.8143 - accuracy: 0.6850 - val_loss: 1.4327 - val_accuracy: 0.4520
Epoch 6/20
45/45 [==============================] - 96s 2s/step - loss: 0.8201 - accuracy: 0.6808 - val_loss: 1.4825 - val_accuracy: 0.4463
Epoch 7/20
45/45 [==============================] - 87s 2s/step - loss: 0.7963 - accuracy: 0.6893 - val_loss: 1.4961 - val_accuracy: 0.5311
Epoch 8/20
45/45 [======

In [ ]:
def eval_model(model, X_test, Y_test):
    score = model.evaluate(X_test, Y_test, verbose=1)
    test_accuracy=print('Test accuracy:', score[1])
    test_loss=print('Test loss:', score[0])
    return test_loss, test_accuracy

test_loss, test_accuracy = eval_model(model, X_test, Y_test)

NameError: ignored

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")